## PyCall

JuliaからPythonパッケージを利用するための仕組み

### Setup

#### Environment
- OS: Ubuntu 18.04 LTS
- Python: `3.7.3` (Miniconda `4.7.10`)
    - Jupyter Notebook: `6.0.0`

#### Installation
少し古い情報を漁ると、REPLで `Pkg.add("PyCall")` というコマンドを叩くと書いてあるが、Julia 1.0以降では、pkgモードでパッケージの追加を行う

```bash
# Julia REPL起動
$ julia

julia> # `]` と打って、pkgモードに移行

# PyCallインストール
(v1.1) pkg> add PyCall
```

In [1]:
# PyCallパッケージを使う
using PyCall

# Pythonバージョン確認
PyCall.pyversion

v"3.7.3"

In [2]:
# Pythonのパスを確認
PyCall.pyprogramname

"/home/user/miniconda3/bin/python3"

In [3]:
# Pythonライブラリのパスを確認
PyCall.libpython

"/home/user/miniconda3/lib/libpython3.7m.so.1.0"

### PyCall経由でPyTorchを使う

- **PyTorch**
    - Python用のDeepLearningフレームワーク
    - Define-by-Run（ニューラルネットワークを定義しながら実行できる）

In [4]:
# @pyimport torch
torch = pyimport("torch")
Variable = torch.autograd.Variable
nn = torch.nn
F = nn.functional

PyObject <module 'torch.nn.functional' from '/home/user/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py'>

In [5]:
@pydef mutable struct Net <: nn.Module

    __init__(self) = begin
        pybuiltin(:super)(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    end

    forward(self, x) = begin
        # Max pooling over a (2, 2) window
        x =  F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
    end

    num_flat_features(self, x) = begin
        size = x.size()[2:end]  # all dimensions except the batch dimension
        num_features = 1
        for s in size
            num_features *= s
        end
        return num_features
    end

end

PyObject <class 'Net'>

In [6]:
net = Net()
println(net)

params = pybuiltin(:list)(net.parameters())
println(length(params))
println(params[1].size())  # conv1's .weight

input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
println(out)

PyObject Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
(6, 1, 5, 5)
PyObject tensor([[-0.0504,  0.0770, -0.0467, -0.0355, -0.0767,  0.0830, -0.0720,  0.0828,
         -0.0056, -0.0718]], grad_fn=<AddmmBackward>)


In [7]:
output = net(input)
target = Variable(torch.arange(1.0, 11.0))  # a dummy target, for example: PyObject tensor(Array{Float})
criterion = nn.MSELoss()

loss = criterion(output, target)
println(loss)

PyObject tensor(38.6390, grad_fn=<MseLossBackward>)


In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

println("conv1.bias.grad before backward")
println(net.conv1.bias.grad === nothing ? "nothing" : net.conv1.bias.grad)

loss.backward()

println("conv1.bias.grad after backward")
println(net.conv1.bias.grad === nothing ? "nothing" : net.conv1.bias.grad)

optim = torch.optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

conv1.bias.grad before backward
nothing
conv1.bias.grad after backward
PyObject tensor([ 0.0361,  0.0454,  0.0251, -0.0240, -0.0288, -0.0490])
